# KASDD TK | a
Siapa tim yang paling banyak memenangkan pertandingan grand final dari sebuah Event

In [2]:
%pip install pandas
%pip install numpy
%pip install scipy
%pip install pandas numpy seaborn scipy plotly matplotlib   

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
REFERENCE = [
    'https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy',
    
    ]

In [4]:
# Import semua library yang dibutuhkan
import pandas as pd
import numpy as np
import scipy.stats as scp
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

In [5]:
# mencari jumlah nilai unik pada kolom yang bertipe kategorik
def check_uniqueness(df):
    for col in df.select_dtypes(include=object).columns:
        print('No. of unique values in column '+col+':')
        print(df[col].value_counts(),'\n')
        
# mengecek nilai null
def check_null(df):
    col_na = df.isnull().sum().sort_values(ascending=True)
    percent = col_na / len(df)
    missing_data = pd.concat([col_na, percent], axis=1, keys=['Total', 'Percent'])

    if (missing_data[missing_data['Total'] > 0].shape[0] == 0):
        print("Tidak ditemukan missing value pada dataset")
    else:
        print(missing_data[missing_data['Total'] > 0])

# mengecek nilai duplikat
def check_duplicates(df):
    dup = df.duplicated()
    print("Jumlah duplikasi data : " + str(dup.sum()))

# mengecek keberadaan outlier (harus numerik)
def check_outlier(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)

    # Menghitung RUB dan RLB.
    IQR = Q3 - Q1
    lower_limit = Q1 - 1.5*IQR
    upper_limit = Q3 + 1.5*IQR

    # Menampilkan banyaknya outlier pada atribut.
    outliers = (df < lower_limit) | (df > upper_limit)
    print ("Outlier pada tiap atribut:")
    print(outliers.sum())
    
    print("Persentase outlier terhadap data per kolom")
    for col in outliers.keys():
        percentage = outliers[col] / len(df)
        print(col)
        print(percentage)
        print()

    return outliers

# Cek nilai outlier & mengembalikan list nama kolom dengan outlier
def outlier_col(dict, total_data):
    outlier_col = []

    print("Persentase outlier terhadap data per kolom")
    for col in dict.keys():
        percentage = dic[col] / total_data
        if 0 < percentage < 1:
            outlier_col.append(col)
    return outlier_col

def drop_outlier(col_to_check, df):
    Q1 = df[col_to_check].quantile(0.25)
    Q3 = df[col_to_check].quantile(0.75)

    # Menghitung RUB dan RLB.
    IQR = Q3 - Q1
    lower_limit = Q1 - 1.5*IQR
    upper_limit = Q3 + 1.5*IQR

    mask = (df[col_to_check] >= lower_limit) & (df[col_to_check] <= upper_limit)
    new_df = df[mask]
    return new_df

In [6]:
# Import dataset
games = pd.read_csv('games.csv')
matches = pd.read_csv('matches.csv')

## Pre-processing

### games.csv

In [7]:
games.columns

Index(['No', 'GameID', 'MatchID', 'Map', 'Team1ID', 'Team2ID', 'Team1',
       'Team2', 'Winner', 'Team1_Eco', 'Team1_SemiEco', 'Team1_SemiBuy',
       'Team1_FullBuy', 'Team1_TotalRounds', 'Team2_Eco', 'Team2_SemiEco',
       'Team2_SemiBuy', 'Team2_FullBuy', 'Team2_TotalRounds'],
      dtype='object')

In [8]:
# check null
# check_null(games)
# games = games.dropna()
# print()
# check_null(games)


In [9]:
# check duplicate
check_duplicates(games)

Jumlah duplikasi data : 0


In [10]:
mask = ['GameID', 'MatchID', 'Map']
games = games[mask]
games.columns

Index(['GameID', 'MatchID', 'Map'], dtype='object')

### matches.csv

In [11]:
matches.columns

Index(['No', 'MatchID', 'Date', 'Patch', 'EventID', 'EventName', 'EventStage',
       'Team1ID', 'Team2ID', 'Team1', 'Team2', 'Team1_MapScore',
       'Team2_MapScore'],
      dtype='object')

In [12]:
# check null
check_null(matches)
matches = matches.dropna()
print()
check_null(matches)

       Total   Percent
Patch    411  0.064209

Tidak ditemukan missing value pada dataset


In [13]:
# check duplicate
check_duplicates(matches)

Jumlah duplikasi data : 0


In [14]:
matches.columns

Index(['No', 'MatchID', 'Date', 'Patch', 'EventID', 'EventName', 'EventStage',
       'Team1ID', 'Team2ID', 'Team1', 'Team2', 'Team1_MapScore',
       'Team2_MapScore'],
      dtype='object')

In [15]:
mask2 = ['MatchID', 'Patch', 'EventID', 'EventName', 'EventStage']
matches = matches[mask2]
matches.columns

Index(['MatchID', 'Patch', 'EventID', 'EventName', 'EventStage'], dtype='object')

## Analyzing

In [16]:
matches['Patch'].value_counts()

Patch
Patch 1.12Minor updates to observing                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         485
Patch 3.0                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [17]:
dfTour = games.merge(matches, left_on='MatchID', right_on='MatchID')
dfTour.columns

Index(['GameID', 'MatchID', 'Map', 'Patch', 'EventID', 'EventName',
       'EventStage'],
      dtype='object')

In [18]:
dfTour['Patch'] = dfTour['Patch'].str.extract(r'(\d)\.\d+').astype(float)
dfTour

GameID  MatchID     Map  Patch  EventID  \
0       60894    62393  Breeze    3.0      826   
1       60895    62393    Bind    3.0      826   
2       60896    62393   Haven    3.0      826   
3       60924    62403  Icebox    3.0      826   
4       60925    62403   Haven    3.0      826   
...       ...      ...     ...    ...      ...   
12173      65       48     TBD    0.0        6   
12174      67       50     TBD    0.0        6   
12175      49       33   Split    0.0        1   
12176      43       29    Bind    0.0        1   
12177      42       28    Bind    0.0        1   

                                               EventName  \
0      Nerd Street Gamers Winter Championship - Regio...   
1      Nerd Street Gamers Winter Championship - Regio...   
2      Nerd Street Gamers Winter Championship - Regio...   
3      Nerd Street Gamers Winter Championship - Regio...   
4      Nerd Street Gamers Winter Championship - Regio...   
...                                                  ...   
12173                             Fnatic Proving Grounds   
12174                             Fnatic Proving Grounds   
12175               T1 x Nerd Street Gamers Invitational   
12176               T1 x Nerd Street Gamers Invitational   
12177               T1 x Nerd Street Gamers Invitational   

                       EventStage  
0        Group Stage: Decider (A)  
1        Group Stage: Decider (A)  
2        Group Stage: Decider (A)  
3        Group Stage: Decider (C)  
4        Group Stage: Decider (C)  
...                           ...  
12173     Tournament: Round of 16  
12174     Tournament: Round of 16  
12175     Tournament: Lower Final  
12176   Tournament: Lower Round 2  
12177  Tournament: Lower Round 1   

[12178 rows x 7 columns]

In [19]:
dfTour['Patch'].value_counts()

Patch
3.0    4347
2.0    4248
1.0    3477
0.0     106
Name: count, dtype: int64

In [20]:
tour_patch0 = dfTour[(dfTour['Patch'] == 0.0) & (dfTour['Map'] != 'TBD')]
tour_patch1 = dfTour[(dfTour['Patch'] == 1.0) & (dfTour['Map'] != 'TBD')]
tour_patch2 = dfTour[(dfTour['Patch'] == 2.0) & (dfTour['Map'] != 'TBD')]
tour_patch3 = dfTour[(dfTour['Patch'] == 3.0) & (dfTour['Map'] != 'TBD')]

tour_patch_list = [tour_patch0, tour_patch1, tour_patch2, tour_patch3]

In [21]:
plt.figure(figsize=(20, 12))
for i in range(4):
    order = tour_patch_list[i]['Map'].value_counts()
    plt.subplot(2, 2, i+1)
    sns.countplot(data=tour_patch_list[i], x='Map', order=order.index, palette='magma')
    plt.title(f'Map paling populer di Tournament Patch {i}.x')
    plt.xlabel('Map')
    plt.ylabel('Jumlah Penggunaan')

plt.show()

C:\Users\vinse\AppData\Local\Temp\ipykernel_20456\1941078249.py:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(data=tour_patch_list[i], x='Map', order=order.index, palette='magma')
C:\Users\vinse\AppData\Local\Temp\ipykernel_20456\1941078249.py:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(data=tour_patch_list[i], x='Map', order=order.index, palette='magma')
C:\Users\vinse\AppData\Local\Temp\ipykernel_20456\1941078249.py:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(data=tour_patch_list[i], x='Map', order=order.index, palette='

In [22]:
sns.countplot(dfTour, x='Map', order=map_played_counts.index, palette='magma').set_ylim(0,4000)
plt.title('Map yang paling sering digunakan dalam Tournament')
plt.xlabel('Map')
plt.ylabel('Jumlah Penggunaan')
plt.show()

NameError: name 'map_played_counts' is not defined

In [ ]:
print("Berdasarkan dataset tersebut, Map yang paling sering digunakan dalam tournament Valorant adalah", map_played_counts.index[0])
print("Dengan jumlah game yang dimainkan di map tersebut sebanyak", map_played_counts.values[0])

Berdasarkan dataset tersebut, Map yang paling sering digunakan dalam tournament Valorant adalah Ascent
Dengan jumlah game yang dimainkan di map tersebut sebanyak 2977
